### SEALION-v3-8B-Instruct LLM run on Aristotle pipeline
### This is run in Runpod/remote Jupyter environment

In [ ]:
!apt update

In [ ]:
!apt install git-lfs

In [ ]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/aisingapore/Llama-SEA-LION-v3-8B-IT LLM_MODELS/Llama-SEA-LION-v3-8B-IT

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"


snapshot_path = "/workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT" ### Change this based on platform and models
#snapshot_path = "aisingapore/Llama-SEA-LION-v3-8B-IT"

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quant (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

### If kernel doesnt recognize
LLM_MODEL=snapshot_path
LOCAL_MODEL_PATH=snapshot_path


In [ ]:
#Test for Max Time generation stopping criteria
from llm_backends import HFBackend
hb = HFBackend(local_model_path=snapshot_path, quantize_4bit=True)
print("Calling short test (3s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=3.0)
print("Result length:", len(res))
print(res[:1000])

print("Calling short test (10s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=10.0)
print("Result length:", len(res))
print(res[:1000])

In [ ]:
#!python run_pipeline.py --dataset_name ProntoQA --sample_pct 0 --batch_size 1
#print("\nFinished translating dataset\n")

In [ ]:
# Change this every process (translate, decompose, search_resolve), different time limit is needed in seconds.
os.environ["LLM_WORKER_MAX_TIME"] = "300"
LLM_WORKER_MAX_TIME=300

# Translation with original prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation --split dev --save_path results_translated_translation/v3/prompts_original --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
# Translation with modified prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_modified --split dev --save_path results_translated_translation/v3/prompts_modified --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
# Translation with refined prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_refine --split dev --save_path results_translated_translation/v3/prompts_refine --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
# Decomposition with refined prompts
#!python decompose_to_cnf.py --data_path results_translated_translation/v3/prompts_refine --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file and_or_decomposer_refine --save_path results_translated_decomposition/v2/prompts_refine --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
#!python negate.py --dataset_name ProntoQA --save_path results_translated_decomposition/v2/prompts_refine --model_name $LLM_MODEL

In [ ]:
# Logic Resolver with original prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file logic_resolver  --save_path results_translated_search_resolve/prompts_original --model_name $LLM_MODEL --batch_num 1 --negation False --search_round 10 --max_new_tokens 6700 

In [ ]:
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file logic_resolver --save_path results_translated_search_resolve/prompts_original --model_name $LLM_MODEL --batch_num 1 --negation True --search_round 10 --max_new_tokens 6700

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_search_resolve/prompts_original --model_name $LLM_MODEL

In [ ]:
# Logic Resolver with modified prompts
!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 10 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_modified  --save_path results_translated_search_resolve/prompts_modified --model_name $LLM_MODEL --batch_num 1 --negation False --search_round 10 --max_new_tokens 6700

^C


Loading raw dataset from results_translated_decomposition/v2/prompts_refine/ProntoQA/LLM_MODEL_trans_decompose_no_negation.json


Traceback (most recent call last):
  File "g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\search_resolve.py", line 640, in <module>
    gpt3_problem_reduction.reasoning_graph_generation()
  File "g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\search_resolve.py", line 325, in reasoning_graph_generation
    raw_dataset = self.load_raw_dataset(self.sample_pct)
  File "g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\search_resolve.py", line 56, in load_raw_dataset
    with open(file_path) as f:
FileNotFoundError: [Errno 2] No such file or directory: 'results_translated_decomposition/v2/prompts_refine/ProntoQA/LLM_MODEL_trans_decompose_no_negation.json'


In [ ]:
!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 10 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_modified  --save_path results_translated_search_resolve/prompts_modified --model_name $LLM_MODEL --batch_num 1 --negation True --search_round 10 --max_new_tokens 6700 

In [ ]:
!python evaluate.py --dataset_name ProntoQA --save_path results_translated_search_resolve/prompts_modified --model_name $LLM_MODEL